# Importação das bibliotecas

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import folium
from folium.plugins import MarkerCluster

# Lendo arquivo

In [30]:
df = pd.read_csv('data/datatran2023.csv', sep=';', encoding='ISO-8859-1')

# ISO-8859-1
# utf-8
# latin1

# Dicionário de Dados

* nb_id
* dt_data_inversa
* tx_dia_semana
* nb_horario
* tx_uf
* br
* km
* municipio
* causa_acidente
* tipo_acidente
* classificacao_acidente
* fase_dia
* sentido_via
* condicao_metereologica
* tipo_pista
* tracado_via
* uso_solo
* pessoas
* mortos
* feridos_leves
* feridos_graves
* ilesos
* ignorados
* feridos
* veiculos
* latitude
* longitude
* regional
* delegacia
* uop

# Tratamento de Dados

In [36]:
# Criando uma nova coluna 'mes' para armazenar o mês correspondente
df['data_inversa'] = pd.to_datetime(df['data_inversa'], format='%Y-%m-%d')  # Certifique-se de ajustar o formato conforme necessário
df['mes'] = df['data_inversa'].dt.month

# Tratando coluna de horário
df.rename(columns={'hora': 'horario'}, inplace=True)
df['horario'].fillna('00:00:00', inplace=True)
df['horario'] = df['horario'].astype(str)
df['hora'] = df['horario'].str[:2]

# Preenchendo nulos de delegacia com uop, quando existente
def extract_from_column(value):
    if pd.isnull(value):
        return None
    else:
        return str(value)[6:]  # Converter para string e pegar a substring a partir do 7º caractere

df['delegacia'] = df.apply(lambda row: extract_from_column(row['uop']) if pd.isnull(row['delegacia']) else row['delegacia'], axis=1)

# Preenchendo nulos de br e km
df[['br', 'km']] = df[['br', 'km']].fillna(0)

# Preenchendo nulos de regional
df['regional'] = df['regional'].fillna('Indefinido')

#Preenchendo nulos de delegacia e uop
df[['delegacia', 'uop']] = df[['delegacia', 'uop']].fillna('Indefinido')

df[['br', 'km']]  = df[['br', 'km']] .astype(str)

# Análise Exploratória

In [37]:
display(df)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,mes,hora
0,496519,2023-01-01,domingo,02:00:00,ES,101.0,114,SOORETAMA,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Simples,Reta,Não,1,0,1,0,0,0,1,1,"-19,09484877","-40,05095848",SPRF-ES,DEL04-ES,UOP01-DEL04-ES,1,02
1,496543,2023-01-01,domingo,03:40:00,SP,116.0,"113,1",TAUBATE,Entrada inopinada do pedestre,Atropelamento de Pedestre,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,Sim,5,1,0,0,0,4,0,1,"-23,0445658","-45,58259814",SPRF-SP,DEL02-SP,UOP02-DEL02-SP,1,03
2,496590,2023-01-01,domingo,01:40:00,MT,163.0,1112,GUARANTA DO NORTE,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Plena Noite,Crescente,Ignorado,Simples,Não Informado,Não,1,0,0,1,0,0,1,1,"-9,70020602","-54,87588757",SPRF-MT,DEL06-MT,UOP03-DEL06-MT,1,01
3,496610,2023-01-01,domingo,10:40:00,PR,376.0,"314,8",ORTIGUEIRA,Velocidade Incompatível,Tombamento,Sem Vítimas,Pleno dia,Crescente,Sol,Dupla,Curva,Não,1,0,0,0,1,0,0,1,"-23,985512","-51,083555",SPRF-PR,DEL07-PR,UOP02-DEL07-PR,1,10
4,496659,2023-01-01,domingo,14:55:00,MG,116.0,"569,4",MANHUACU,Acumulo de água sobre o pavimento,Colisão frontal,Com Vítimas Feridas,Pleno dia,Decrescente,Chuva,Simples,Não Informado,Não,3,0,0,2,1,0,2,2,"-20,10007457","-42,17884091",SPRF-MG,DEL06-MG,UOP03-DEL06-MG,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38403,541876,2023-03-13,segunda-feira,18:10:00,ES,101.0,"293,6",CARIACICA,Ausência de sinalização,Atropelamento de Pedestre,Com Vítimas Feridas,Anoitecer,Crescente,Céu Claro,Múltipla,Reta,Sim,2,0,0,1,1,0,1,1,"-20,32415948","-40,3952903",SPRF-ES,DEL01-ES,UOP01-DEL01-ES,3,18
38404,542057,2023-07-29,sábado,10:00:00,GO,60.0,"164,8",GOIANIA,Velocidade Incompatível,Colisão traseira,Com Vítimas Fatais,Pleno dia,Crescente,Sol,Múltipla,Não Informado,Sim,3,1,0,0,2,0,0,2,"-16,7036","-49,342",SPRF-GO,DEL01-GO,UOP02-DEL01-GO,7,10
38405,542058,2023-07-08,sábado,18:50:00,PR,116.0,1,CAMPINA GRANDE DO SUL,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Decrescente,Ignorado,Dupla,Reta,Não,4,0,4,0,0,0,4,1,"-25,06368403","-48,57051896",SPRF-PR,DEL01-PR,UOP02-DEL01-PR,7,18
38406,542112,2023-06-22,quinta-feira,05:25:00,MG,381.0,"746,5",TRES CORACOES,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Dupla,Não Informado,Não,54,0,1,0,53,0,1,1,"-21,618761","-45,262445",SPRF-MG,DEL16-MG,UOP02-DEL16-MG,6,05


## Quantidade de linhas e colunas

In [38]:
df.shape

(38408, 32)

## Tipos das colunas

In [39]:
df.dtypes

id                                 int64
data_inversa              datetime64[ns]
dia_semana                        object
horario                           object
uf                                object
br                                object
km                                object
municipio                         object
causa_acidente                    object
tipo_acidente                     object
classificacao_acidente            object
fase_dia                          object
sentido_via                       object
condicao_metereologica            object
tipo_pista                        object
tracado_via                       object
uso_solo                          object
pessoas                            int64
mortos                             int64
feridos_leves                      int64
feridos_graves                     int64
ilesos                             int64
ignorados                          int64
feridos                            int64
veiculos        

## Nulos por variável

In [40]:
df.isnull().sum()

id                        0
data_inversa              0
dia_semana                0
horario                   0
uf                        0
br                        0
km                        0
municipio                 0
causa_acidente            0
tipo_acidente             0
classificacao_acidente    0
fase_dia                  0
sentido_via               0
condicao_metereologica    0
tipo_pista                0
tracado_via               0
uso_solo                  0
pessoas                   0
mortos                    0
feridos_leves             0
feridos_graves            0
ilesos                    0
ignorados                 0
feridos                   0
veiculos                  0
latitude                  0
longitude                 0
regional                  0
delegacia                 0
uop                       0
mes                       0
hora                      0
dtype: int64

## Estatísticas Gerais

In [67]:
df.describe(include='all')
# include='all'

C:\Users\LILIAN\AppData\Local\Temp\ipykernel_144\3448564409.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,mes,hora
count,6163.000000,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163,6163.000000,6163.000000,6163.000000,6163.000000,6163.000000,6163.000000,6163.000000,6163.000000,6163,6163,6163,6163,6163,6163.000000,6163
unique,NaN,212,7,667,10,1,2245,201,66,16,3,4,2,8,3,10,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2993,2993,9,27,59,NaN,24
top,NaN,2023-06-25 00:00:00,domingo,18:00:00,SP,116.0,230,GUARULHOS,Reação tardia ou ineficiente do condutor,Colisão traseira,Com Vítimas Feridas,Pleno dia,Crescente,Céu Claro,Dupla,Reta,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-23,52251783","-46,5855702",SPRF-SP,DEL01-SP,UOP01-DEL01-SP,NaN,18
freq,NaN,48,1032,113,1920,6163,43,421,1101,1366,4790,3256,3192,3755,2890,3812,3229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,45,1920,641,641,NaN,483
first,NaN,2023-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2023-07-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,518643.385202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.436313,0.071232,0.891449,0.229271,1.086484,0.157878,1.120720,1.695765,NaN,NaN,NaN,NaN,NaN,4.136946,NaN
std,12247.354808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.126240,0.303001,1.060823,0.647698,1.666873,0.458145,1.218713,0.771170,NaN,NaN,NaN,NaN,NaN,1.971577,NaN
min,496506.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
25%,508154.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,3.000000,NaN


# Análise Temporal

In [21]:
# Ocorrências agrupadas por mês
value_counts = df['mes'].value_counts()
sorted_counts = value_counts.sort_index()

display(sorted_counts)

1    5250
2    4746
3    5589
4    5599
5    5622
6    5702
7    5900
Name: mes, dtype: int64

In [22]:
# Ocorrências agrupadas por dia da semana
df['dia_semana'].value_counts()

domingo          6503
sábado           6262
sexta-feira      5826
segunda-feira    5422
quinta-feira     5042
quarta-feira     4752
terça-feira      4601
Name: dia_semana, dtype: int64

In [23]:
# Ocorrências agrupadas por horário
value_counts = df['hora'].value_counts()
sorted_counts = value_counts.sort_index()

display(sorted_counts)

00     801
01     687
02     605
03     700
04     828
05    1179
06    1771
07    2387
08    1961
09    1554
10    1606
11    1690
12    1630
13    1613
14    1825
15    1925
16    2115
17    2580
18    3012
19    2446
20    1742
21    1453
22    1242
23    1056
Name: hora, dtype: int64

## Análise Geográfica

In [24]:
# Ocorrências agrupadas por estados
df_uf = df['uf'].value_counts()
df_uf.head(10)

MG    5047
SC    4504
PR    4041
RJ    3082
RS    2943
SP    2710
BA    2092
GO    1770
PE    1675
ES    1299
Name: uf, dtype: int64

In [25]:
# Ocorrências agrupadas por municípios
df_municipios = df['municipio'].value_counts()
df_municipios.head(10)

BRASILIA                563
GUARULHOS               475
SAO JOSE                385
DUQUE DE CAXIAS         375
PALHOCA                 359
BETIM                   341
RECIFE                  329
CURITIBA                323
SERRA                   286
SAO JOSE DOS PINHAIS    265
Name: municipio, dtype: int64

## Insights de Negócios

In [26]:
display(df)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,mes,hora
0,496519,2023-01-01,domingo,02:00:00,ES,101.0,114,SOORETAMA,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Simples,Reta,Não,1,0,1,0,0,0,1,1,"-19,09484877","-40,05095848",SPRF-ES,DEL04-ES,UOP01-DEL04-ES,1,02
1,496543,2023-01-01,domingo,03:40:00,SP,116.0,"113,1",TAUBATE,Entrada inopinada do pedestre,Atropelamento de Pedestre,Com Vítimas Fatais,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,Sim,5,1,0,0,0,4,0,1,"-23,0445658","-45,58259814",SPRF-SP,DEL02-SP,UOP02-DEL02-SP,1,03
2,496590,2023-01-01,domingo,01:40:00,MT,163.0,1112,GUARANTA DO NORTE,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Plena Noite,Crescente,Ignorado,Simples,Não Informado,Não,1,0,0,1,0,0,1,1,"-9,70020602","-54,87588757",SPRF-MT,DEL06-MT,UOP03-DEL06-MT,1,01
3,496610,2023-01-01,domingo,10:40:00,PR,376.0,"314,8",ORTIGUEIRA,Velocidade Incompatível,Tombamento,Sem Vítimas,Pleno dia,Crescente,Sol,Dupla,Curva,Não,1,0,0,0,1,0,0,1,"-23,985512","-51,083555",SPRF-PR,DEL07-PR,UOP02-DEL07-PR,1,10
4,496659,2023-01-01,domingo,14:55:00,MG,116.0,"569,4",MANHUACU,Acumulo de água sobre o pavimento,Colisão frontal,Com Vítimas Feridas,Pleno dia,Decrescente,Chuva,Simples,Não Informado,Não,3,0,0,2,1,0,2,2,"-20,10007457","-42,17884091",SPRF-MG,DEL06-MG,UOP03-DEL06-MG,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38403,541876,2023-03-13,segunda-feira,18:10:00,ES,101.0,"293,6",CARIACICA,Ausência de sinalização,Atropelamento de Pedestre,Com Vítimas Feridas,Anoitecer,Crescente,Céu Claro,Múltipla,Reta,Sim,2,0,0,1,1,0,1,1,"-20,32415948","-40,3952903",SPRF-ES,DEL01-ES,UOP01-DEL01-ES,3,18
38404,542057,2023-07-29,sábado,10:00:00,GO,60.0,"164,8",GOIANIA,Velocidade Incompatível,Colisão traseira,Com Vítimas Fatais,Pleno dia,Crescente,Sol,Múltipla,Não Informado,Sim,3,1,0,0,2,0,0,2,"-16,7036","-49,342",SPRF-GO,DEL01-GO,UOP02-DEL01-GO,7,10
38405,542058,2023-07-08,sábado,18:50:00,PR,116.0,1,CAMPINA GRANDE DO SUL,Ausência de reação do condutor,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Decrescente,Ignorado,Dupla,Reta,Não,4,0,4,0,0,0,4,1,"-25,06368403","-48,57051896",SPRF-PR,DEL01-PR,UOP02-DEL01-PR,7,18
38406,542112,2023-06-22,quinta-feira,05:25:00,MG,381.0,"746,5",TRES CORACOES,Condutor Dormindo,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Céu Claro,Dupla,Não Informado,Não,54,0,1,0,53,0,1,1,"-21,618761","-45,262445",SPRF-MG,DEL16-MG,UOP02-DEL16-MG,6,05


In [27]:
# Km das rodovias que mais possuem acidentes
grouped = df.groupby(['br', 'km'])['causa_acidente'].count().reset_index()
grouped_sorted = grouped.sort_values(by='causa_acidente', ascending=False)

display(grouped_sorted.head(10))

,br,km,causa_acidente
0,0.0,0,92
3323,116.0,230,43
3314,116.0,229,40
3265,116.0,219,39
2077,101.0,70,38
733,101.0,206,35
3285,116.0,223,32
3303,116.0,227,32
1308,101.0,322,30
3276,116.0,220,29


In [28]:
# Ocorrências agrupadas por causa dos acidentes
df_municipios = df['causa_acidente'].value_counts()
df_municipios.head(10)

Reação tardia ou ineficiente do condutor                     5550
Ausência de reação do condutor                               5188
Acessar a via sem observar a presença dos outros veículos    3626
Velocidade Incompatível                                      2478
Condutor deixou de manter distância do veículo da frente     2438
Manobra de mudança de faixa                                  2277
Ingestão de álcool pelo condutor                             2095
Demais falhas mecânicas ou elétricas                         1564
Transitar na contramão                                       1347
Condutor Dormindo                                            1163
Name: causa_acidente, dtype: int64

## Classificação do acidente

In [41]:
# Ocorrências agrupadas por classificação do acidente
df_municipios = df['classificacao_acidente'].value_counts()
df_municipios.head(10)

Com Vítimas Feridas    29350
Sem Vítimas             6268
Com Vítimas Fatais      2790
Name: classificacao_acidente, dtype: int64

In [53]:
# Rodovias e kilometragens com maior índice de acidentes com vítimas fatais
classificacao_desejada = 'Com Vítimas Fatais'
df_filtrado = df[df['classificacao_acidente'] == classificacao_desejada]

# Agrupar e contar acidentes por rodovia (br) e quilômetro (km)
grouped = df_filtrado.groupby(['br', 'km'])['causa_acidente'].count().reset_index()

# Ordenar em ordem decrescente pelo número de acidentes
grouped_sorted = grouped.sort_values(by='causa_acidente', ascending=False)

# Exibir as 10 primeiras linhas
print(grouped_sorted.head(10))

         br     km  causa_acidente
525   116.0    229               6
1384  242.0    852               3
519   116.0    222               3
1504  277.0    114               3
495   116.0    188               3
1230  230.0    115               3
512   116.0    212               3
665   116.0    512               3
2118  381.0  169,9               2
806   153.0    113               2


# Gerando mapa de acidentes

In [29]:
# m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=13)

# # Adicionar um cluster de marcadores para melhorar a visualização de múltiplos pontos
# marker_cluster = MarkerCluster().add_to(m)

# # Adicionar marcadores para cada ponto de acidente
# for idx, row in df.iterrows():
#     folium.Marker([row['latitude'], row['longitude']]).add_to(marker_cluster)

# # Exibir o mapa
# m.save('mapa_acidentes.html')